In [48]:
import numpy as np
import math
from random import random
from functools import reduce
from queue import PriorityQueue, LifoQueue
from collections import namedtuple

In [92]:
PROBLEM_SIZE = 40
NUM_SETS = 20
SETS = tuple(
    np.array([random() < 0.3 for _ in range(PROBLEM_SIZE)])
    for _ in range(NUM_SETS)
)
State = namedtuple('State', ['taken', 'not_taken'])

# for i in SETS:
#         print(i)

In [93]:
assert goal_check(State(set(range(NUM_SETS)), set())), "Problem is not solvable"

In [98]:
def goal_check(state):
    return all(reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    ))  

def special_sets():
    # this function evaluets which are the tiles (the indexes) which are less common to be covered
    # then it procedes to take the set which has the highest amount of TRUE comprending also the rare tile
    # it take the set from the non_taken
    # Instead of starting from an empty set() we initialize the problem with the set that most probably will be
    # fundamental in our solution
    state = State(set(), set(range(NUM_SETS)))
    tiles_distribution = np.zeros(PROBLEM_SIZE)

    for i in state.not_taken:
        tiles_distribution = tiles_distribution + SETS[i]

    scarse_index = np.argmin(tiles_distribution)
    starting_point = [-1, -1] # [set, score]

    for i in [x for x in state.not_taken if SETS[x][scarse_index] == True]:
        score = sum(SETS[i])
        if score > starting_point[1]:
            starting_point = [i, score]

    if starting_point[0] != -1:
        state = State(
            state.taken ^ {starting_point[0]},
            state.not_taken ^ {starting_point[0]},
        )

    return state 


# OLD H that wasn't suitable for A*
def h(state):
    # Gives an estimation on how far the current frontier is from the goal state
    return PROBLEM_SIZE - sum(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        ))

def h1(state):
    # This function works as follow:
    # 1. it looks among the not_taken sets and find which one has the highest number N of TRUE tiles
    # 2. it calculates how many FALSE tiles we still have to cover (let's call it M)
    # 3. it returns N/M, which is an the number of sets we need to take (at least, optimistically) to reach the goal state
    # 4. example: i still miss 5 tiles M=5, we find a set among not taken with N=2 => it returns ceil(5/2)=3, because
    #    optimistaclly we will just need 3 tiles to solve the problem 
    _sorted = sorted(state.not_taken, key=lambda i: sum(SETS[i]), reverse=True)

    return math.ceil((PROBLEM_SIZE - sum(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )))
        /
        sum(SETS[_sorted[0]]))

def h2(state):
    # This heuristic is an improvement of h1:
    # Instead of counting the number N of TRUE tiles in the non_taken sets, we count only the TRUE tiles
    # that are usefull to improve our current state, i.e. the tiles that are precisely missing still in our state.
    
    current_state = reduce(np.logical_or, [SETS[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)]), )
    already_taken = sum(current_state)
    _or = [np.logical_or(SETS[i], current_state) for i in state.not_taken]
    _sorted = sorted(_or, key=lambda _set: sum(_set), reverse=True)
    best = sum(_sorted[0]) - already_taken

    # practially speaking, best is equal to then number of TRUE tiles of our best promising set in the not_taken
    # s.t. each TRUE is still missing in the current state

    if best == 0:
        return 0

    return math.ceil(already_taken/best)

def g(state):
    # Gives 34the actual distance from the start state (in terms of number of node)
    return len(state.taken)

def f(state):
    return g(state) + h2(state)

In [99]:
frontier = PriorityQueue()
# calculate here the special sets => starting not from the empty set but from a strategic point
# state = State(set(), set(range(NUM_SETS)))
state = special_sets()
frontier.put((f(state), state))

counter = 0
_, current_state = frontier.get()
while not goal_check(current_state):
    counter += 1
    for action in current_state.not_taken:
        new_state = State(
            current_state.taken ^ {action},
            current_state.not_taken ^ {action},
        )
        frontier.put((f(new_state), new_state))
    _, current_state = frontier.get()

print(
    f"Solved in {counter:,} steps ({len(current_state.taken)} sets)"
)

print(
    f"The current sate is: {current_state}"
)


Solved in 6,219 steps (6 sets)
The current sate is: State(taken={2, 8, 11, 13, 15, 19}, not_taken={0, 1, 3, 4, 5, 6, 7, 9, 10, 12, 14, 16, 17, 18})
